In [1]:
import numpy as np
from tensorflow.keras.preprocessing import image
from PIL import Image
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import Model
from tensorflow.keras.layers import Input, Dense, Dropout, Flatten, Conv2D, MaxPooling2D, GlobalAveragePooling2D, concatenate
from tensorflow.keras import optimizers
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, f1_score
from sklearn.utils import shuffle

In [2]:
path_to_accuracy = 'accuracies_our_model5.npy'
path_to_f1score = 'f1score_our_model5.npy'
path_to_test_indices = 'test_indices.npy'
path_to_train_indices = 'train_indices.npy'
path_to_X = 'X.npy'
path_to_Y = 'Y.npy'

fold_idx = 3

In [3]:
global_height = 128
global_width = 128
input_channels = 3
num_classes = 3

In [4]:
def fire(x, squeeze, expand):
    y  = Conv2D(filters=squeeze, kernel_size=1, activation='relu', padding='same')(x)
    y1 = Conv2D(filters=expand//2, kernel_size=1, activation='relu', padding='same')(y)
    y3 = Conv2D(filters=expand//2, kernel_size=3, activation='relu', padding='same')(y)
    return concatenate([y1, y3])

def fire_module(squeeze, expand):
    return lambda x: fire(x, squeeze, expand)

def get_model():
    x = Input(shape = [global_height, global_width, input_channels])

    y = Conv2D(kernel_size=3, filters=32, padding='same', use_bias=True, activation='relu')(x)
    y = fire_module(16, 32)(y)
    y = MaxPooling2D(pool_size=2)(y)
    y = fire_module(24, 48)(y)
    y = MaxPooling2D(pool_size=2)(y)

    y = Dropout(0.1)(y)

    y = fire_module(24, 48)(y)
    y = MaxPooling2D(pool_size=2)(y)
    y = fire_module(16, 32)(y)
    y = GlobalAveragePooling2D()(y)
    y = Dense(num_classes, activation='softmax')(y)

    model = Model(x, y)
    
    return model

In [5]:
train_indices = np.load(path_to_train_indices)
test_indices = np.load(path_to_test_indices)
accuracies = np.load(path_to_accuracy)
f1scores = np.load(path_to_f1score)

train_index = train_indices[fold_idx]
test_index = test_indices[fold_idx]

In [6]:
def get_patches(img, new_height, new_width):
  height, width, depth = img.shape 
  patches = []

  for r in range(0, height, new_height):
    for c in range(0, width, new_width):
      if r+new_height <= height and c+new_width <= width:
        curr_patch = img[r: r+new_height, c: c+new_width, :]
        patches.append(curr_patch)
        
  return patches

In [7]:
def get_train_test():
    
    X = np.load(path_to_X)
    Y = np.load(path_to_Y)
    
    print('X: ', X.shape)
    print('Y: ', Y.shape)
    print()
    
    X_train = X[train_index]
    Y_train = Y[train_index]
    
    X_test = X[test_index]
    Y_test = Y[test_index]
    
    return X_train, X_test, Y_train, Y_test

In [8]:
def getCompleteDataset():
    X_train, X_test, Y_train, Y_test = get_train_test()
    
    print('X Train:', X_train.shape)
    print('Y Train:', Y_train.shape)
    
    print('X Test:', X_test.shape)
    print('Y Test:', Y_test.shape)
    
    print()
    
    X_train_patches = []
    Y_train_patches = []
    
    for i in range(0, len(X_train), 1):
        patches = get_patches(X_train[i], global_height, global_width)
        numOfPatches = len(patches)
        for p in patches:
            X_train_patches.append(p)
            Y_train_patches.append(Y_train[i])
    
    numOfPatches = len(get_patches(X_train[0], global_height, global_width))
    print('One image creates no. of patches: ', numOfPatches)
    print()
    
    X_train_patches = np.array(X_train_patches)
    Y_train_patches = np.array(Y_train_patches)
    
    # Shuffling the training patch data alongwith their label
    X_train_patches, Y_train_patches = shuffle(X_train_patches, Y_train_patches, random_state = 0)
    
    return X_train_patches, Y_train_patches, X_test, Y_test

In [9]:
X_train_patches, Y_train_patches, X_test, Y_test = getCompleteDataset()
print('Training Data in patches: ',X_train_patches.shape, Y_train_patches.shape)
print('One training patch size: ', X_train_patches[0].shape, Y_train_patches[0].shape)
print('Original test data: ', X_test.shape, Y_test.shape)

X:  (450, 1040, 1388, 3)
Y:  (450, 3)

X Train: (360, 1040, 1388, 3)
Y Train: (360, 3)
X Test: (90, 1040, 1388, 3)
Y Test: (90, 3)

One image creates no. of patches:  80

Training Data in patches:  (28800, 128, 128, 3) (28800, 3)
One training patch size:  (128, 128, 3) (3,)
Original test data:  (90, 1040, 1388, 3) (90, 3)


In [10]:
!free -m

              total        used        free      shared  buff/cache   available
Mem:          30147        8398       20674         608        1073       20766
Swap:             0           0           0


In [11]:
def model_run():
    
    model = get_model()
    
    model.compile(optimizers.Adam(lr=0.0005),loss="categorical_crossentropy",metrics=["accuracy"])
    
    history = model.fit(X_train_patches, Y_train_patches, epochs = 30)
    
    y_test_pred = []

    for i in range(len(X_test)):
        patches = get_patches(X_test[i], global_height, global_width)
        patches = np.array(patches)
        curr_pred = model.predict(patches)

        curr_pred_single = np.argmax(curr_pred, axis=1)
        max_occurring_class = np.bincount(curr_pred_single).argmax()

        y_test_pred.append(max_occurring_class)

    y_test_pred = np.array(y_test_pred)
    y_test = np.argmax(Y_test, axis=1)
    
    confusion_matrix_var = confusion_matrix(y_test, y_test_pred)
    accuracy_var = accuracy_score(y_test, y_test_pred)
    classification_report_var = classification_report(y_test, y_test_pred)
    
    f1score_var = f1_score(y_test, y_test_pred, average='weighted')
    
    return accuracy_var, confusion_matrix_var, classification_report_var, f1score_var
    

In [12]:
accuracy_var, confusion_matrix_var, classification_report_var, f1score_var = model_run()

Train on 28800 samples
Epoch 1/30
28800/28800 [==============================] - 34s 1ms/sample - loss: 0.8644 - accuracy: 0.5692
Epoch 2/30
28800/28800 [==============================] - 29s 1ms/sample - loss: 0.5524 - accuracy: 0.7515
Epoch 3/30
28800/28800 [==============================] - 29s 1ms/sample - loss: 0.4350 - accuracy: 0.8194
Epoch 4/30
28800/28800 [==============================] - 29s 1ms/sample - loss: 0.3610 - accuracy: 0.8533
Epoch 5/30
28800/28800 [==============================] - 29s 1ms/sample - loss: 0.3094 - accuracy: 0.8780
Epoch 6/30
28800/28800 [==============================] - 29s 1ms/sample - loss: 0.2721 - accuracy: 0.8962
Epoch 7/30
28800/28800 [==============================] - 29s 1ms/sample - loss: 0.2506 - accuracy: 0.9024
Epoch 8/30
28800/28800 [==============================] - 29s 1ms/sample - loss: 0.2249 - accuracy: 0.9145
Epoch 9/30
28800/28800 [==============================] - 29s 1ms/sample - loss: 0.2138 - accuracy: 0.9177s - loss: 0.213

In [13]:
print(confusion_matrix_var)
print(classification_report_var)

[[30  0  0]
 [ 1 28  1]
 [ 1  0 29]]
              precision    recall  f1-score   support

           0       0.94      1.00      0.97        30
           1       1.00      0.93      0.97        30
           2       0.97      0.97      0.97        30

    accuracy                           0.97        90
   macro avg       0.97      0.97      0.97        90
weighted avg       0.97      0.97      0.97        90



In [14]:
# Append the accuracy to the 'accuracies' variable and save it in a file
# Similarly for F1 score

accuracies = np.append(accuracies, accuracy_var)
np.save(path_to_accuracy, accuracies)

f1scores = np.append(f1scores, f1score_var)
np.save(path_to_f1score, f1scores)

In [15]:
# Print all the accuracies accumulated till now
accuracies = np.load(path_to_accuracy)
for i in range(len(accuracies)):
    print('For fold number: ', i)
    print('Accuracy: ', accuracies[i])
    print('\n'*2)

print('Average Accuracy: ', np.mean(accuracies))

For fold number:  0
Accuracy:  0.9888888888888889



For fold number:  1
Accuracy:  0.9444444444444444



For fold number:  2
Accuracy:  0.9777777777777777



For fold number:  3
Accuracy:  0.9666666666666667



Average Accuracy:  0.9694444444444444


In [16]:
# Print all the f1 scores accumulated till now
f1scores = np.load(path_to_f1score)
for i in range(len(f1scores)):
    print('For fold number: ', i)
    print('F1 Score: ', f1scores[i])
    print('\n'*2)

print('Average F1 score: ', np.mean(f1scores))

For fold number:  0
F1 Score:  0.9888858016115588



For fold number:  1
F1 Score:  0.9441893258374847



For fold number:  2
F1 Score:  0.9779478767997085



For fold number:  3
F1 Score:  0.9666419478432827



Average F1 score:  0.9694162380230086
